---
title: "Feature Exploration"
date: 2021-02-24
type: technical_note
draft: false
---

---

## <span style="color:#ff5f27;">🧑🏻‍🏫 HSFS feature exploration </span>

In this notebook we are going to walk through how to use the HSFS library to explore feature groups and features in the Hopsworks Feature Store. 

A key component of the Hopsworks feature store is to enable sharing and re-using of features across models and use cases. As such, the HSFS libraries allows user to join features from different feature groups and use them to create training datasets.
Features can be taken also from different feature stores (projects) as long as the user running the notebook has the read access to those.

![Join](../images/join.svg "Join")

As for the [feature_engineering](./feature_engineering.ipynb) notebook, the first step is to establish a connection with the feature store and retrieve the project feature store handle.

In [1]:
import hsfs

# Create a connection
connection = hsfs.connection()

# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


---

## <span style="color:#ff5f27;"> 🕵🏻‍♂️ Explore feature groups </span>

You can interact with the feature groups as if they were Spark dataframe. A feature group object has a `show()` method, to show `n` number of lines, and a `read()` method to read the content of the feature group in a Spark dataframe.

The first step to do any operation on a feature group is to get its handle from the feature store. The `get_feature_group` method accepts the name of the feature group and an optional parameter with the version you want to select. If you do not provide a version, the APIs will default to version 1

In [2]:
sales_fg = fs.get_or_create_feature_group(
    name = "sales_fg",
    version = 1
)

In [3]:
sales_fg.select(['date','weekly_sales','is_holiday', 'sales_last_30_days_store_dep', 'sales_last_365_days_store_dep']).show(5)

2022-06-20 13:51:09,946 INFO: USE `basics_featurestore`
2022-06-20 13:51:10,805 INFO: SELECT `fg0`.`date` `date`, `fg0`.`weekly_sales` `weekly_sales`, `fg0`.`is_holiday` `is_holiday`, `fg0`.`sales_last_30_days_store_dep` `sales_last_30_days_store_dep`, `fg0`.`sales_last_365_days_store_dep` `sales_last_365_days_store_dep`
FROM `basics_featurestore`.`sales_fg_1` `fg0`


,date,weekly_sales,is_holiday,sales_last_30_days_store_dep,sales_last_365_days_store_dep
0,1304035200000,3659.65,0,149274.40,1802200.44
1,1296777600000,2934.61,0,154996.76,471965.34
2,1328832000000,18208.24,1,631611.49,6589537.79
3,1346371200000,10230.28,0,416341.08,0.00
4,1311292800000,17140.02,0,607928.45,3963133.37


In [4]:
sales_df = sales_fg.read()
sales_df[sales_df.store == 20].head()

2022-06-20 13:51:18,948 INFO: USE `basics_featurestore`
2022-06-20 13:51:19,823 INFO: SELECT `fg0`.`store` `store`, `fg0`.`dept` `dept`, `fg0`.`date` `date`, `fg0`.`weekly_sales` `weekly_sales`, `fg0`.`is_holiday` `is_holiday`, `fg0`.`sales_last_30_days_store_dep` `sales_last_30_days_store_dep`, `fg0`.`sales_last_30_days_store` `sales_last_30_days_store`, `fg0`.`sales_last_90_days_store_dep` `sales_last_90_days_store_dep`, `fg0`.`sales_last_90_days_store` `sales_last_90_days_store`, `fg0`.`sales_last_180_days_store_dep` `sales_last_180_days_store_dep`, `fg0`.`sales_last_180_days_store` `sales_last_180_days_store`, `fg0`.`sales_last_365_days_store_dep` `sales_last_365_days_store_dep`, `fg0`.`sales_last_365_days_store` `sales_last_365_days_store`
FROM `basics_featurestore`.`sales_fg_1` `fg0`


,store,dept,date,weekly_sales,is_holiday,sales_last_30_days_store_dep,sales_last_30_days_store,sales_last_90_days_store_dep,sales_last_90_days_store,sales_last_180_days_store_dep,sales_last_180_days_store,sales_last_365_days_store_dep,sales_last_365_days_store
70,20,94,1342742400000,68495.69,0,1982312.56,1982312.56,5677882.27,5677882.27,11014154.51,11014154.51,31409255.20,31409255.20
71,20,12,1308268800000,10485.09,0,191750.50,191750.50,977080.59,977080.59,3319720.12,3319720.12,10552557.44,10552557.44
85,20,3,1348185600000,11109.08,0,486562.46,486562.46,1417725.91,1417725.91,5472437.62,5472437.62,16621469.81,16621469.81
145,20,80,1279238400000,41604.13,0,1173650.61,1173650.61,3499311.12,3499311.12,6490119.60,6490119.60,12913043.36,12913043.36
211,20,79,1299801600000,39220.63,0,1137264.43,1137264.43,2683499.33,2683499.33,4715535.39,4715535.39,18667704.91,18667704.91


In [5]:
print(f'⛳️ Type: {type(sales_df)}')

⛳️ Type: <class 'pandas.core.frame.DataFrame'>


You can also inspect the metadata of the feature group. You can, for instance, show the features the feature group is made of and if they are primary or partition keys:

In [6]:
print("Name: {}".format(sales_fg.name))
print("Description: {}".format(sales_fg.description))
print("Features:")
features = sales_fg.features
for feature in features:
    print("{:<60} \t Primary: {} \t Partition: {}".format(feature.name, feature.primary, feature.partition))

Name: sales_fg
Description: Sales related features
Features:
store                                                        	 Primary: True 	 Partition: False
dept                                                         	 Primary: True 	 Partition: False
date                                                         	 Primary: True 	 Partition: False
weekly_sales                                                 	 Primary: False 	 Partition: False
is_holiday                                                   	 Primary: False 	 Partition: False
sales_last_30_days_store_dep                                 	 Primary: False 	 Partition: False
sales_last_30_days_store                                     	 Primary: False 	 Partition: False
sales_last_90_days_store_dep                                 	 Primary: False 	 Partition: False
sales_last_90_days_store                                     	 Primary: False 	 Partition: False
sales_last_180_days_store_dep                                	 Primar

If you are interested only in a subset of features, you can use the `select()` method on the feature group object to select a list of features. The `select()` behaves like a feature group, as such, you can call the `.show()` or `.read()` methods on it.

In [7]:
sales_fg.select(['store', 'dept', 'weekly_sales']).show(5)

2022-06-20 13:51:34,057 INFO: USE `basics_featurestore`
2022-06-20 13:51:34,925 INFO: SELECT `fg0`.`store` `store`, `fg0`.`dept` `dept`, `fg0`.`weekly_sales` `weekly_sales`
FROM `basics_featurestore`.`sales_fg_1` `fg0`


,store,dept,weekly_sales
0,32,71,3659.65
1,21,67,2934.61
2,31,10,18208.24
3,8,1,10230.28
4,1,23,17140.02


---

## <span style="color:#ff5f27;"> 👮🏼‍♀️ Filter Feature Groups </span>

If you do not want to read your feature group into a dataframe, you can also filter directly on a `FeatureGroup` object. Applying a filter to a feature group returns a `Query` object which can subsequently be used to be further joined with other feature groups or queries, or it can be materialized as a training dataset.

In [8]:
sales_fg.select(['store', 'dept', 'weekly_sales']).filter(sales_fg.weekly_sales >= 50000).show(5)

2022-06-20 13:51:40,887 INFO: USE `basics_featurestore`
2022-06-20 13:51:41,804 INFO: SELECT `fg0`.`store` `store`, `fg0`.`dept` `dept`, `fg0`.`weekly_sales` `weekly_sales`
FROM `basics_featurestore`.`sales_fg_1` `fg0`
WHERE `fg0`.`weekly_sales` >= 50000


,store,dept,weekly_sales
0,30,38,60223.39
1,19,16,56018.16
2,13,93,55090.05
3,9,38,83424.12
4,27,72,67459.34


Conjunctions of filters can be constructed using the Python Bitwise Operators `|` and `&` which replace the regular binary operators when working with feature groups and filters.

In [9]:
sales_fg.select(['store', 'dept', 'weekly_sales']).filter((sales_fg.weekly_sales >= 50000) & (sales_fg.dept == 2)).show(5)

2022-06-20 13:51:45,228 INFO: USE `basics_featurestore`
2022-06-20 13:51:46,205 INFO: SELECT `fg0`.`store` `store`, `fg0`.`dept` `dept`, `fg0`.`weekly_sales` `weekly_sales`
FROM `basics_featurestore`.`sales_fg_1` `fg0`
WHERE `fg0`.`weekly_sales` >= 50000 AND `fg0`.`dept` = 2


,store,dept,weekly_sales
0,19,2,61912.39
1,14,2,82661.34
2,11,2,54084.66
3,4,2,87181.89
4,19,2,50399.44


---

## <span style="color:#ff5f27;"> 👷🏼‍♂️ Join Features and Feature Groups </span>

HSFS provides an API similar to Pandas to join feature groups together and to select features from different feature groups.
The easies query you can write is by selecting all the features from a feature group and join them with all the features of another feature group.

You can use the `select_all()` method of a feature group to select all its features. HSFS relies on the Hopsworks feature store to identify which features of the two feature groups to use as joining condition. 
If you don't specify anything, Hopsworks will use the largest matching subset of primary keys with the same name.

In the example below, `sales_fg` has `store`, `dept` and `date` as composite primary key while `exogenous_fg` has only `store` and `date`. So Hopsworks will set as joining condition `store` and `date`.

In [10]:
sales_fg = fs.get_or_create_feature_group(
    name = 'sales_fg',
    version = 1
)

exogenous_fg = fs.get_or_create_feature_group(
    name = 'exogenous_fg',
    version = 1
)

query = sales_fg.select_all().join(exogenous_fg.select_all())

You can use the query object to create training datasets (see training dataset notebook). You can inspect the query generated by calling the `to_string()` method on it.

In [11]:
print(query.to_string())

SELECT `fg1`.`store` `store`, `fg1`.`dept` `dept`, `fg1`.`date` `date`, `fg1`.`weekly_sales` `weekly_sales`, `fg1`.`is_holiday` `is_holiday`, `fg1`.`sales_last_30_days_store_dep` `sales_last_30_days_store_dep`, `fg1`.`sales_last_30_days_store` `sales_last_30_days_store`, `fg1`.`sales_last_90_days_store_dep` `sales_last_90_days_store_dep`, `fg1`.`sales_last_90_days_store` `sales_last_90_days_store`, `fg1`.`sales_last_180_days_store_dep` `sales_last_180_days_store_dep`, `fg1`.`sales_last_180_days_store` `sales_last_180_days_store`, `fg1`.`sales_last_365_days_store_dep` `sales_last_365_days_store_dep`, `fg1`.`sales_last_365_days_store` `sales_last_365_days_store`, `fg0`.`temperature` `temperature`, `fg0`.`fuel_price` `fuel_price`, `fg0`.`markdown1` `markdown1`, `fg0`.`markdown2` `markdown2`, `fg0`.`markdown3` `markdown3`, `fg0`.`markdown4` `markdown4`, `fg0`.`markdown5` `markdown5`, `fg0`.`cpi` `cpi`, `fg0`.`unemployment` `unemployment`, `fg0`.`is_holiday` `is_holiday`, CASE WHEN `fg0`.`a

As for the feature groups, you can call the `show()` method to inspect the data before generating a training dataset from it. Or you can call the `read()` method to get a Spark DataFrame with the result of the query and apply additional transformations to it.

In [12]:
query.show(5)

2022-06-20 13:51:50,028 INFO: USE `basics_featurestore`
2022-06-20 13:51:50,849 INFO: SELECT `fg1`.`store` `store`, `fg1`.`dept` `dept`, `fg1`.`date` `date`, `fg1`.`weekly_sales` `weekly_sales`, `fg1`.`is_holiday` `is_holiday`, `fg1`.`sales_last_30_days_store_dep` `sales_last_30_days_store_dep`, `fg1`.`sales_last_30_days_store` `sales_last_30_days_store`, `fg1`.`sales_last_90_days_store_dep` `sales_last_90_days_store_dep`, `fg1`.`sales_last_90_days_store` `sales_last_90_days_store`, `fg1`.`sales_last_180_days_store_dep` `sales_last_180_days_store_dep`, `fg1`.`sales_last_180_days_store` `sales_last_180_days_store`, `fg1`.`sales_last_365_days_store_dep` `sales_last_365_days_store_dep`, `fg1`.`sales_last_365_days_store` `sales_last_365_days_store`, `fg0`.`temperature` `temperature`, `fg0`.`fuel_price` `fuel_price`, `fg0`.`markdown1` `markdown1`, `fg0`.`markdown2` `markdown2`, `fg0`.`markdown3` `markdown3`, `fg0`.`markdown4` `markdown4`, `fg0`.`markdown5` `markdown5`, `fg0`.`cpi` `cpi`, `f

,store,dept,date,weekly_sales,is_holiday,sales_last_30_days_store_dep,sales_last_30_days_store,sales_last_90_days_store_dep,sales_last_90_days_store,sales_last_180_days_store_dep,...,fuel_price,markdown1,markdown2,markdown3,markdown4,markdown5,cpi,unemployment,is_holiday,appended_feature
0,7,67,1335484800000,3100.23,0,128353.04,128353.04,368801.46,368801.46,469529.79,...,3.927,819.48,NaN,45.94,49.82,1209.89,197.664546,8.090,0,10.0
1,41,4,1331251200000,29320.67,0,955751.92,955751.92,2749310.67,2749310.67,4651795.12,...,3.286,15255.04,285.89,7.74,3203.61,3889.43,196.963160,6.589,0,10.0
2,27,44,1272585600000,4235.89,0,230722.72,230722.72,846650.03,846650.03,1567698.75,...,3.042,NaN,NaN,NaN,NaN,NaN,135.917083,8.058,0,10.0
3,11,81,1339113600000,14540.75,0,392018.44,392018.44,1150766.82,1150766.82,2402243.25,...,3.452,11984.13,191.40,78.65,2768.31,6358.77,225.252146,6.664,0,10.0
4,6,96,1319155200000,26781.65,0,960483.58,960483.58,2709414.12,2709414.12,10796825.64,...,3.353,NaN,NaN,NaN,NaN,NaN,219.074017,6.551,0,10.0


As for the `show()` and `read()` method of the feature group, even in the case of a query you can specify against which storage to run the query.

---

## <span style="color:#ff5f27;"> 💼 Select only a subset of features </span>

You can replace the `select_all()` method with the `select([])` method to be able to select only a subset of features from a feature group you want to join:

In [13]:
query = sales_fg.select(['store', 'dept', 'weekly_sales'])\
                .join(exogenous_fg.select(['fuel_price']))
query.show(5)

2022-06-20 13:52:46,041 INFO: USE `basics_featurestore`
2022-06-20 13:52:46,947 INFO: SELECT `fg1`.`store` `store`, `fg1`.`dept` `dept`, `fg1`.`weekly_sales` `weekly_sales`, `fg0`.`fuel_price` `fuel_price`
FROM `basics_featurestore`.`sales_fg_1` `fg1`
INNER JOIN `basics_featurestore`.`exogenous_fg_1` `fg0` ON `fg1`.`store` = `fg0`.`store` AND `fg1`.`date` = `fg0`.`date`


,store,dept,weekly_sales,fuel_price
0,7,67,3100.23,3.927
1,41,4,29320.67,3.286
2,27,44,4235.89,3.042
3,11,81,14540.75,3.452
4,6,96,26781.65,3.353


---

## <span style="color:#ff5f27;"> 💈 Overwrite the joining key </span>

If your feature groups don't have a primary key, or if they have different names or if you want to overwrite the joining key, you can pass it as a parameter of the join.

As in Pandas, if the feature has the same name on both feature groups, then you can use the `on=[]` paramter. If they have different names, then you can use the `left_on=[]` and `right_on=[]` paramters:

In [14]:
query = sales_fg.select(['store', 'dept', 'weekly_sales'])\
                .join(exogenous_fg.select(['fuel_price']), on=['date'])
query.show(5)

2022-06-20 13:53:16,607 INFO: USE `basics_featurestore`
2022-06-20 13:53:17,440 INFO: SELECT `fg1`.`store` `store`, `fg1`.`dept` `dept`, `fg1`.`weekly_sales` `weekly_sales`, `fg0`.`fuel_price` `fuel_price`
FROM `basics_featurestore`.`sales_fg_1` `fg1`
INNER JOIN `basics_featurestore`.`exogenous_fg_1` `fg0` ON `fg1`.`date` = `fg0`.`date`


,store,dept,weekly_sales,fuel_price
0,11,85,1823.7,2.735
1,11,85,1823.7,3.070
2,11,85,1823.7,3.162
3,11,85,1823.7,2.830
4,11,85,1823.7,2.735


### <span style="color:#ff5f27;">⛓ Overwriting the join type </span>



By default, the join type between two feature groups is `INNER JOIN`. You can overwrite this behavior by passing the `join_type` parameter to the join method. Valid types are: `INNER, LEFT, RIGHT, FULL, CROSS, LEFT_SEMI_JOIN, COMMA`

In [15]:
query = sales_fg.select(['store', 'dept', 'weekly_sales'])\
                .join(exogenous_fg.select(['fuel_price']), join_type="left")

print(query.to_string())

SELECT `fg1`.`store` `store`, `fg1`.`dept` `dept`, `fg1`.`weekly_sales` `weekly_sales`, `fg0`.`fuel_price` `fuel_price`
FROM `basics_featurestore`.`sales_fg_1` `fg1`
LEFT JOIN `basics_featurestore`.`exogenous_fg_1` `fg0` ON `fg1`.`store` = `fg0`.`store` AND `fg1`.`date` = `fg0`.`date`


---

## <span style="color:#ff5f27;"> 🧲 Join mulitple feature groups </span>

You can concatenate as many feature gropus as you wish. In the example below the order of execution will be:

    (`sales_fg` <> `store_fg`) <> `exogenous_fg`

The join paramers you pass in each `join()` method call apply to that specific join. This means that you can concatenate left and right joins.
Please be aware that currently HSFS **does not support** nested join such as: 

    `sales_fg` <> (`store_fg` <> `exogenous_fg`)

In [16]:
store_fg = fs.get_or_create_feature_group(
    name = "store_fg",
    version = 1
)

query = sales_fg.select_all()\
                .join(store_fg.select_all())\
                .join(exogenous_fg.select(['fuel_price', 'unemployment', 'cpi']))

print(query.to_string())

SELECT `fg2`.`store` `store`, `fg2`.`dept` `dept`, `fg2`.`date` `date`, `fg2`.`weekly_sales` `weekly_sales`, `fg2`.`is_holiday` `is_holiday`, `fg2`.`sales_last_30_days_store_dep` `sales_last_30_days_store_dep`, `fg2`.`sales_last_30_days_store` `sales_last_30_days_store`, `fg2`.`sales_last_90_days_store_dep` `sales_last_90_days_store_dep`, `fg2`.`sales_last_90_days_store` `sales_last_90_days_store`, `fg2`.`sales_last_180_days_store_dep` `sales_last_180_days_store_dep`, `fg2`.`sales_last_180_days_store` `sales_last_180_days_store`, `fg2`.`sales_last_365_days_store_dep` `sales_last_365_days_store_dep`, `fg2`.`sales_last_365_days_store` `sales_last_365_days_store`, `fg0`.`type` `type`, `fg0`.`size` `size`, `fg0`.`dept` `dept`, `fg1`.`fuel_price` `fuel_price`, `fg1`.`unemployment` `unemployment`, `fg1`.`cpi` `cpi`
FROM `basics_featurestore`.`sales_fg_1` `fg2`
INNER JOIN `basics_featurestore`.`store_fg_1` `fg0` ON `fg2`.`store` = `fg0`.`store`
INNER JOIN `basics_featurestore`.`exogenous_fg_1

### <span style="color:#ff5f27;"> 🔱 Use Joins together with Filters </span>

It is possible to use filters in any of the subqueries of a joined query.

In [17]:
query = sales_fg.select_all()\
                .join(store_fg.select_all())\
                .join(exogenous_fg.select(['fuel_price', 'unemployment', 'cpi']).filter(exogenous_fg.fuel_price <= 2.7)) \
                .filter(sales_fg.weekly_sales >= 50000)

print(query.to_string())

SELECT `fg2`.`store` `store`, `fg2`.`dept` `dept`, `fg2`.`date` `date`, `fg2`.`weekly_sales` `weekly_sales`, `fg2`.`is_holiday` `is_holiday`, `fg2`.`sales_last_30_days_store_dep` `sales_last_30_days_store_dep`, `fg2`.`sales_last_30_days_store` `sales_last_30_days_store`, `fg2`.`sales_last_90_days_store_dep` `sales_last_90_days_store_dep`, `fg2`.`sales_last_90_days_store` `sales_last_90_days_store`, `fg2`.`sales_last_180_days_store_dep` `sales_last_180_days_store_dep`, `fg2`.`sales_last_180_days_store` `sales_last_180_days_store`, `fg2`.`sales_last_365_days_store_dep` `sales_last_365_days_store_dep`, `fg2`.`sales_last_365_days_store` `sales_last_365_days_store`, `fg0`.`type` `type`, `fg0`.`size` `size`, `fg0`.`dept` `dept`, `fg1`.`fuel_price` `fuel_price`, `fg1`.`unemployment` `unemployment`, `fg1`.`cpi` `cpi`
FROM `basics_featurestore`.`sales_fg_1` `fg2`
INNER JOIN `basics_featurestore`.`store_fg_1` `fg0` ON `fg2`.`store` = `fg0`.`store`
INNER JOIN `basics_featurestore`.`exogenous_fg_1

---

## <span style="color:#ff5f27;"> 🔮 Free hand query </span>

With HSFS you are free of writing skipping entirely the Hopsworks query constructor and write your own query. This functionality can be useful if you need to express more complex queries for your use case. `fs.sql` returns a Spark Dataframe.

In [18]:
fs.sql("SELECT * FROM `store_fg_1`").head()

2022-06-20 13:58:08,689 INFO: USE `basics_featurestore`
2022-06-20 13:58:09,381 INFO: SELECT * FROM `store_fg_1`


,store_fg_1.store,store_fg_1.type,store_fg_1.size,store_fg_1.dept,store_fg_1._hoodie_record_key,store_fg_1._hoodie_partition_path,store_fg_1._hoodie_commit_time,store_fg_1._hoodie_file_name,store_fg_1._hoodie_commit_seqno
0,26,A,152513,76,26,,20220620130034998,2daa2664-4907-4932-af25-c089474b316b-0_0-29-66...,20220620130034998_0_1
1,1,A,151315,77,1,,20220620130034998,2daa2664-4907-4932-af25-c089474b316b-0_0-29-66...,20220620130034998_0_2
2,21,B,140167,77,21,,20220620130034998,2daa2664-4907-4932-af25-c089474b316b-0_0-29-66...,20220620130034998_0_3
3,17,B,93188,76,17,,20220620130034998,2daa2664-4907-4932-af25-c089474b316b-0_0-29-66...,20220620130034998_0_4
4,30,C,42988,64,30,,20220620130034998,2daa2664-4907-4932-af25-c089474b316b-0_0-29-66...,20220620130034998_0_5


---
## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the following notebook we will use our feature groups to create a dataset we can train a model on.